#### asyncio: don't forget to await everything - it's almost a generator

In [ ]:
import asyncio

async def worker(name, queue, to_consume, semaphore):
    async with semaphore:
        print(f'worker {name} consuming {to_consume}')
        await asyncio.sleep(1)
        l = len(to_consume)
        if l == 1:
            return
        await queue.put(to_consume[:l//2])
        await queue.put(to_consume[l//2:])
    
async def coordinator():
    queue = asyncio.Queue()
    semaphore = asyncio.Semaphore(5)
    await queue.put('abcdefghijklmn')
    while True:
        to_consume = await queue.get()
        if to_consume:
            asyncio.create_task(worker('', queue, to_consume, semaphore))

await coordinator()

#### Threads: Remember your locks

In [ ]:
import time
from concurrent.futures import ThreadPoolExecutor
from queue import Queue

def worker(to_consume, queue):
    print(f'consuming {to_consume}')
    time.sleep(1)
    l = len(to_consume)
    if l == 1:
        return
    queue.put(to_consume[:l//2])
    queue.put(to_consume[l//2:])

def coordinator():
    pool = ThreadPoolExecutor(5)
    q = Queue()
    q.put('abcdefghijklmn')
    while True:
        to_consume = q.get()
        pool.submit(worker, to_consume, q)

coordinator()